In [30]:
from plyfile import PlyData, PlyElement
import pandas as pd
import numpy as np
import os.path as osp

pcd_fp = "./westbahnhof_temp.ply"
osp.exists(pcd_fp)

pcd = pd.DataFrame(PlyData.read(pcd_fp).elements[0].data)
pos = pcd[['x','y','z']].to_numpy()
rgb = pcd[['red','green','blue']].to_numpy()
field_names = [s for s in pcd.columns if s.startswith('scalar') and 'labels' in s]
# max_label = [int(pcd[s].max().item()) for s in field_names]

label_joined = pcd[field_names].to_numpy(dtype=np.int64)
unique, inds, inv, counts = np.unique(label_joined,axis=0, return_counts=True, return_inverse=True, return_index=True)




# cumsum = 0
# for fn, mx in zip(field_names, max_label):
#     y = pcd[fn].copy() + cumsum
#     cumsum += mx






In [42]:
print(f"Found {inv.max()} unique classes")

Found 10 unique classes


In [ ]:
# save as separate ply files in _temp
for i in range(inv.max()+1):
    PlyData([PlyElement.describe(pcd[inv == i].to_records(index=False),'vertex')]).write(f"./_temp/{i:02}.ply")

# Joining ply

In [77]:
# get first pcd seg
from glob import glob
for cls_i, pcd_fp in enumerate(glob("./_temp/*.ply")):
    print(osp.basename(pcd_fp))
    p = pd.DataFrame(PlyData.read(pcd_fp).elements[0].data)
    break
p.columns

addon.ply


Index(['x', 'y', 'z', 'red', 'green', 'blue', 'scalar_Intensity',
       'scalar_Return_Number', 'scalar_Number_Of_Returns'],
      dtype='object')

In [78]:

names_keep = ['x', 'y', 'z', 'red', 'green', 'blue', 'scalar_Intensity']
names_new = [n.replace('scalar_','') for n in names_keep]
names_map = {old:new for old,new in zip(names_keep,names_new)}
names_map

{'x': 'x',
 'y': 'y',
 'z': 'z',
 'red': 'red',
 'green': 'green',
 'blue': 'blue',
 'scalar_Intensity': 'Intensity'}

In [79]:
from glob import glob
import yaml

out_name = 'niebelungen_flatLabels'
cls_names = []
pcds_seg = []
schema = {}
for cls_i, pcd_fp in enumerate(glob("./_temp/*.ply")):
    print(osp.basename(pcd_fp))
    p = pd.DataFrame(PlyData.read(pcd_fp).elements[0].data)
    p = p[names_keep]
    p = p.rename(columns=names_map)
    p['labels'] = np.full(p.shape[0], cls_i,dtype=np.int32)
    if p[['red','green','blue']].to_numpy().max() > 1.0:
        p[['red','green','blue']] /=255

    pcds_seg.append(p)
    schema[cls_i] = osp.basename(pcd_fp).replace('.ply','')


pcds_seg    = pd.concat(pcds_seg,axis=0,ignore_index=True)

pcd_seg     = PlyData([PlyElement.describe(pcds_seg.to_records(index=False),'vertex')])
pcd_seg.write(f'./{out_name}.ply')
with open(f'{out_name}.yaml', 'w') as f:
    yaml.dump(schema,f)
    

addon.ply
attatchment.ply
beam.ply
column.ply
deck.ply
fundament.ply
pipe.ply
railing.ply
roadway.ply
road_barrier.ply
road_marking.ply
scan_artefact.ply
